In [157]:
import langchain
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate
from langchain.utilities import SerpAPIWrapper
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, SystemMessage, HumanMessage
import re

# How to add memory:
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

In [158]:
langchain.debug = True
langchain.verbose = True

In [159]:
# Define which tools the agent can use to answer user queries
search = SerpAPIWrapper(serpapi_api_key='')
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [180]:
# Set up the base template
template = """Complete the objective as best you can. You have access to the following tools:

{tools}

----

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

These were previous tasks you completed:

Begin!

Question: {input}
{agent_scratchpad}"""

In [181]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [SystemMessage(content=formatted), [HumanMessage(content="This is a test")]]

In [182]:
# How to add memory:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [183]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"],
)

In [172]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "final answer:" in llm_output.lower():
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.lower().split("final answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [173]:
output_parser = CustomOutputParser()

In [184]:
agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

In [179]:
memory.buffer

[]

In [191]:
llm = ChatOpenAI(temperature=0)
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names,
    agent_kwargs=agent_kwargs,
    memory=memory
)

In [192]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [193]:
agent_executor.run("Hi my name is bob.")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Hi my name is bob."
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "intermediate_steps": [],
  "stop": [
    "\nObservation:"
  ],
  "input": "Hi my name is bob."
}
{'input': 'Hi my name is bob.', 'agent_scratchpad': '', 'tools': 'Search: useful for when you need to answer questions about current events', 'tool_names': 'Search'}
[chain/error] [1:chain:AgentExecutor > 2:chain:LLMChain] s] Chain run errored with error:
"ValidationError(model='ChatPromptValue', errors=[{'loc': ('messages', 1), 'msg': 'value is not a valid dict', 'type': 'type_error.dict'}])"
[chain/error] [1:chain:AgentExecutor] s] Chain run errored with error:
"ValidationError(model='ChatPromptValue', errors=[{'loc': ('messages', 1), 'msg': 'value is not a valid dict', 'type': 'type_error.dict'}])"


ValidationError: 1 validation error for ChatPromptValue
messages -> 1
  value is not a valid dict (type=type_error.dict)